In [1]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py

In [2]:
%matplotlib widget

#%reload_ext autoreload
#%autoreload 2

### Discharge data

In [3]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
fsamp = 5000
def func1_scale(i):
    bb = step_cal_files[i]
        
    freq2 = np.fft.rfftfreq(len(bb.electrode_data[0]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.electrode_data[0]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    print(phi_tmp2,np.std(bb.electrode_data[0])*np.sqrt(2))

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [4]:
step_cal_dir = r'/data/new_trap/20210622/bead1/Discharge/FineDischarge/'
step_cal_files = load_dir(step_cal_dir, start_file=300)

100%|██████████| 280/280 [00:01<00:00, 160.84it/s]

580  files in folder
280  files loaded


In [5]:
amps3 = []
for i in range(len(step_cal_files)):
    print(i,'/',len(step_cal_files))
    amps3.append(func1_scale(i))

0 / 280
[0.08190331] 9.996283880412916
1 / 280
[0.08190909] 9.996281304225382
2 / 280
[0.08189878] 9.996281537733502
3 / 280
[0.08188683] 9.996288002004473
4 / 280
[0.08188301] 9.996283855239406
5 / 280
[0.0818812] 9.996280896041402
6 / 280
[0.08188132] 9.996280781109695
7 / 280
[0.08187602] 9.996281466488083
8 / 280
[0.08187166] 9.996283836062767
9 / 280
[0.08186824] 9.99628049649982
10 / 280
[0.08185259] 9.99628122742368
11 / 280
[0.08184229] 9.996278292760515
12 / 280
[0.08184375] 9.9962872679154
13 / 280
[0.08185011] 9.996282311106809
14 / 280
[0.08185688] 9.996282232848417
15 / 280
[0.08187448] 9.99628608262862
16 / 280
[0.0818733] 9.996285608676523
17 / 280
[0.08187297] 9.996284865507036
18 / 280
[0.08186803] 9.996282915362116
19 / 280
[0.08185791] 9.996282562713075
20 / 280
[0.08185112] 9.996280282430336
21 / 280
[0.08186028] 9.996284879184568
22 / 280
[0.08187175] 9.99628085579499
23 / 280
[0.08186014] 9.996283020975962
24 / 280
[0.08186719] 9.996287090219038
25 / 280
[0.081872

207 / 280
[0.08191724] 9.996286803289863
208 / 280
[0.08191292] 9.996282726737947
209 / 280
[0.08190771] 9.99627836823018
210 / 280
[0.08189533] 9.996286220251093
211 / 280
[0.08189578] 9.996282530728573
212 / 280
[0.08189087] 9.996279481465574
213 / 280
[0.0818991] 9.996283213603508
214 / 280
[0.08190371] 9.996283979268693
215 / 280
[0.08190964] 9.996278217408733
216 / 280
[0.08190812] 9.996281835959412
217 / 280
[0.08189192] 9.996278896802057
218 / 280
[0.08189368] 9.996275926505898
219 / 280
[0.08190671] 9.996279128442351
220 / 280
[0.08179691] 9.99627468897426
221 / 280
[0.08182042] 9.99627516703645
222 / 280
[0.08180523] 9.99627521228304
223 / 280
[0.08181146] 9.99627824130788
224 / 280
[0.08183009] 9.996278267050043
225 / 280
[0.08180829] 9.996275926682001
226 / 280
[0.08182235] 9.996279022607434
227 / 280
[0.08182145] 9.996281920292775
228 / 280
[0.08182219] 9.99628283021442
229 / 280
[0.08184658] 9.996279959458805
230 / 280
[0.08186107] 9.996278940572699
231 / 280
[0.08185479] 

In [6]:
scale = np.abs(np.mean(amps3[180:200]))
scaleY = scale*-1.0502
print("Scale is ", scaleY)
bb = step_cal_files[-1]
voltage = np.std(bb.electrode_data[0])*np.sqrt(2)
print("voltage: ", voltage)

Scale is  -3.542952205750171e-05
voltage:  9.996282495738035


In [237]:
# amps = np.array(amps)
amps3 = np.array(amps3)
_,ax = plt.subplots()
# scale = 326.26

# Perrinf
ax.scatter(range(len(amps3)), amps3/scale, label='fit phase')
# ax.scatter(range(len(amps3)), amps3/scale, label='fit fixed-phases')

ax.set(xlabel='dataset', ylabel='Electron charges', ylim=(-1.2,6.5), xlim=(160,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [258]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
phase_calib = -0.0563
# phase_calib = -0.0442

def get_amp(i):
    
    fname = r'/data/new_trap/20210622/bead1/Neutrality2/Relaxation_20210708/200V_single/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)


    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phase_calib, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}
    
#     fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phase_calib, 'sigma': 1,
#                   'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'error_sigma': 1,
#                   'errordef': 1,
#                   'limit_phi': [-2 * np.pi, 2 * np.pi], 
#                   'limit_A': [-1, 1], 'limit_sigma': [0,1000],
#                   'print_level': 0, 'fix_f': True, 'fix_phi': True, 'fix_sigma': False}
    
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.y2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    fit_kwargs['phi'] = phase_calib+np.pi/2
    fit_kwargs['f'] = drive_freq*2
    b, a = signal.butter(3, [2.*(drive_freq*2-bandwidth/2.)/fsamp, 2.*(drive_freq*2+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.y2)[::decimate]
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0], -1*m2_tmp.values[0]

In [259]:
i=0
get_amp(i)/scaleY

array([-0.07166299, -0.00853857])

In [279]:
scaleY

-3.542952205750171e-05

In [265]:
i = 1000
fname = r'/data/new_trap/20210622/bead1/Neutrality2/Relaxation_20210708/200V_single/Discharge_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
voltage0_200_1 = np.std(bb.electrode_data[0])*np.sqrt(2)*200
voltage1_200_1 = np.std(bb.electrode_data[1])*np.sqrt(2)*200
bias2 = np.mean(bb.electrode_data[0])*200
print("voltages, bias: ", voltage0_200_1, voltage1_200_1)

voltages, bias:  199.9148713430895 0.0060555832587817415


In [39]:
voltage

9.996282495738035

In [266]:
amps_data_200V_single = np.array(Parallel(n_jobs=40)(delayed(get_amp)(i) for i in tqdm(range(2000))))


100%|██████████| 2000/2000 [00:24<00:00, 83.26it/s]

In [253]:
filtered_data_200V = [a for a,b in zip(amps_data_200V_2[:,0],amps_data_200V_2[:,1]) if (b<1e-5)&(b>-1e-5)]
len(filtered_data_200V), len(amps_data_200V_2[:,0])

(13166, 13200)

In [278]:
# data0 = amps_data_20V[:,0]/scaleY*voltage/voltage0_200*10
# data0 = amps_data_100V[:,1]/scaleY*voltage/voltage0_200*2/1.18
# data0 = amps_data_150V[:,1]/scaleY*voltage/voltage0_200*1.5/1.18
data0 = amps_data_200V/scaleY*voltage/voltage0_200
# data0 = amps_data_40V[:,0]/scaleY*voltage/voltage0_200*5
# data0 = amps_data_40V[:,1]/scaleY*voltage/voltage0_200*5/1.18
# data0 = amps_data_200V_1[:,0]/scaleY*voltage/voltage0_200
# data0 = amps_data_200V_2/scaleY*voltage/voltage0_200
# data0 = amps_data_200V_2_2[:,0]/scaleY*voltage/voltage0_200 ## with sigma for noise
# data0 = filtered_data_200V/scaleY*voltage/voltage0_200 ## 2-harmonic cut

# data0 = amps_data_200V_single/scaleY*voltage/voltage0_200
# dataPlus=np.array(data0[:,0])
# dataMinus=np.array(data0[:,1])/1.18

dataPlus=np.array(data0[::2,0])
dataMinus=np.array(data0[1::2,1])
dataPlus_B=np.array(data0[::2,1])/1.18
dataMinus_B=np.array(data0[1::2,1])/1.18

_, ax= plt.subplots(figsize=(9.2,4))
ax.scatter(np.arange(len(dataMinus)), dataPlus, label='Y-axis, plus, 200V, XZ-spin-46kHz, with-height-fb')
ax.scatter(np.arange(len(dataMinus_B)), dataMinus, label='Y-axis, minus, 200V, XZ-spin-46kHz, with-height-fb')
ax.plot(np.arange(0,len(dataMinus)), np.arange(0,len(dataMinus))*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
ax.set(ylim=(-0.03,0.03))
print('mean: ', np.mean(dataPlus_B), 'std: ', np.std(dataPlus_B)/np.sqrt(len(dataPlus)))
print('mean: ', np.mean(dataMinus_B), 'std: ', np.std(dataMinus_B)/np.sqrt(len(dataMinus)))
print('A mean: ', np.mean(dataPlus+dataMinus*0.85), 'A std: ', np.std(dataPlus+dataMinus*0.85)/np.sqrt(len(dataMinus)))
print('B mean: ', np.mean(dataPlus_B-dataMinus_B*0.85*0.85), 
      'B std: ', np.std(dataPlus_B-dataMinus_B*0.85*0.85)/np.sqrt(len(dataMinus_B)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean:  0.031422808752704694 std:  0.00021075644860574534
mean:  -0.00012268236788897654 std:  0.0002086260346250291
A mean:  -0.00387862754495518 A std:  0.00038882125318231254
B mean:  0.03151144676350448 B std:  0.0002641785386074703


In [248]:
_,ax = plt.subplots()
_ = ax.hist(dataMinus, bins=40, range=(-0.03,0.03))
_ = ax.hist(dataPlus, bins=40, range=(-0.03,0.03), alpha=0.4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [251]:
_,ax = plt.subplots()
ax.scatter(amps_data_200V_2[::4,0],amps_data_200V_2[2::4,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [242]:
_,ax = plt.subplots(1,2,figsize=(9.2,4), sharey=True, sharex=True)
ax[0].set(xlim=(-1e-5,1e-5), ylim=(-1e-5,1e-5))
ax[0].scatter(amps_data_200V_2[::4,0], amps_data_200V_2[::4,1])
# ax[0].scatter(amps_data_200V_2_sideband[::4,0], amps_data_200V_2_sideband[::4,1], alpha=0.4)
ax[1].scatter(amps_data_200V_2[2::4,0], amps_data_200V_2[2::4,1])
# ax[1].scatter(amps_data_200V_2_sideband[2::4,0], amps_data_200V_2_sideband[2::4,1], alpha=0.4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [244]:
binning = int(25000/500)

means = np.array([np.mean(dataPlus[j*500:(j+1)*500]) for j in range(binning)])
stds = np.array([np.std(dataPlus[j*500:(j+1)*500]) for j in range(binning)])

means2 = np.array([np.mean(dataMinus[j*500:(j+1)*500]) for j in range(binning)])
stds2 = np.array([np.std(dataMinus[j*500:(j+1)*500]) for j in range(binning)])

_,ax = plt.subplots(figsize=(9.2,4))
ax.errorbar(np.arange(0,25000,500), means, yerr=stds/np.sqrt(500),fmt='o')
ax.errorbar(np.arange(0,25000,500), means2, yerr=stds2/np.sqrt(500), fmt='o')
ax.errorbar(np.arange(0,25000,500), means+means2*0.857, yerr=stds2/np.sqrt(500), fmt='o')
ax.plot(np.arange(0,2500,10), np.arange(0,2500,10)*0, 'r--')
ax.set(xlabel='dataset', ylabel=r'$\epsilon$')
print(np.mean(dataPlus), np.mean(np.array(dataPlus)+np.array(dataMinus)*0.857))
print(np.std(np.array(dataPlus)+np.array(dataMinus)*0.857)/np.sqrt(500))
print(means+means2*0.85)
print(stds)
print(stds2)

/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-0.00012869681962166386 -0.00019190595291223046
0.0005430554888475616
[ 0.00056603 -0.00034765 -0.00023327 -0.00072458  0.00019401 -0.00060287
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan]
[0.00893506 0.00891729 0.00897102 0.00907103 0.00925645 0.00940754
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        